In [30]:
import pandas as pd
import os
import glob
from pathlib import Path

In [2]:
df=pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv")

In [3]:
locs = pd.read_csv(open('../../data-locations/locations.csv', 'r'))
locs = locs[locs['location']!='US'].astype({'location':'int64'})

In [4]:
st_pop = df[[ 'Province_State', 'Population']].groupby(
    'Province_State').sum().reset_index().rename(
    columns={'Province_State':'location_name'}
).merge(
    locs, how='left',on='location_name'
).dropna(subset=['abbreviation'])
pop_df = df[df['FIPS'].notna()][['FIPS', 'Province_State', 'Admin2','Population']].astype({'FIPS':'int64'}).rename(columns={'FIPS':'location'})
pop_df = pd.concat([pop_df, st_pop]).astype({'location':'int64'})[['location', 'Province_State', 'Admin2','Population']]

In [5]:
df_pop = locs.merge(
    pop_df, how='left', on='location'
)[['abbreviation', 'location', 'location_name', 'Population']].drop_duplicates()

,abbreviation,location,location_name,Population
0,AL,1,Alabama,4903185.0
1,AK,2,Alaska,731545.0
2,AZ,4,Arizona,7278717.0
3,AR,5,Arkansas,3017804.0
4,CA,6,California,39512223.0
...,...,...,...,...
3198,NaN,56037,Sweetwater County,42343.0
3199,NaN,56039,Teton County,23464.0
3200,NaN,56041,Uinta County,20226.0
3201,NaN,56043,Washakie County,7805.0


In [27]:
us_tot = st_pop[st_pop['location'] < 56]['Population'].sum()
df_pop.append({'abbreviation':'US', 'location':-1, 'location_name':'United States', 'Population':us_tot}, ignore_index=True)
# df_pop['location'] = df_pop['location'].apply(lambda x: '{0:0>2}'.format(x))

,abbreviation,location,location_name,Population
0,AL,01,Alabama,4903185.0
1,AK,02,Alaska,731545.0
2,AZ,04,Arizona,7278717.0
3,AR,05,Arkansas,3017804.0
4,CA,06,California,39512223.0
...,...,...,...,...
3195,NaN,56039,Teton County,23464.0
3196,NaN,56041,Uinta County,20226.0
3197,NaN,56043,Washakie County,7805.0
3198,NaN,56045,Weston County,6927.0


In [28]:
df_pop.to_csv(open('../../data-locations/populations.csv','w'), index=False)

In [37]:
model = glob.glob('../../data-processed/**/*.csv')[0]

In [38]:
model_df = pd.read_csv(open(model, 'r'))

In [47]:
pop = pd.read_csv(open('../../data-locations/populations.csv','r'))
model_df 

,forecast_date,target,target_end_date,location,type,quantile,value
0,2020-08-09,1 wk ahead cum death,2020-08-15,-1,quantile,0.010,167564.510140
1,2020-08-09,2 wk ahead cum death,2020-08-22,-1,quantile,0.010,172096.857640
2,2020-08-09,3 wk ahead cum death,2020-08-29,-1,quantile,0.010,175813.829262
3,2020-08-09,4 wk ahead cum death,2020-09-05,-1,quantile,0.010,179240.292465
4,2020-08-09,1 wk ahead cum death,2020-08-15,-1,quantile,0.025,167785.814940
...,...,...,...,...,...,...,...
25597,2020-08-09,2 wk ahead inc case,2020-08-22,56045,point,NaN,0.000000
25598,2020-08-09,3 wk ahead inc case,2020-08-29,56045,point,NaN,0.000000
25599,2020-08-09,1 wk ahead inc case,2020-08-15,56045,quantile,0.500,0.000000
25600,2020-08-09,2 wk ahead inc case,2020-08-22,56045,quantile,0.500,0.000000


In [53]:
model_df['location'].replace('US', -1, inplace=True)
model_df = model_df.astype({'location':'int64'})

In [54]:
pop

,abbreviation,location,location_name,Population
0,US,-1,United States,328149707.0
1,AL,1,Alabama,4903185.0
2,AK,2,Alaska,731545.0
3,AZ,4,Arizona,7278717.0
4,AR,5,Arkansas,3017804.0
...,...,...,...,...
3195,NaN,56037,Sweetwater County,42343.0
3196,NaN,56039,Teton County,23464.0
3197,NaN,56041,Uinta County,20226.0
3198,NaN,56043,Washakie County,7805.0


In [55]:
d = model_df.merge(pop[['location', 'Population']], on='location', how='left')
d.to_csv(open('temp.csv','w'), index=False)

In [63]:
import numpy as np
d[d['value'] >= d['Population']]

,forecast_date,target,target_end_date,location,type,quantile,value,Population


In [65]:
pop = pd.read_csv(open('../../data-locations/populations.csv','r'))
models = glob.glob('../../data-processed/**/*.csv')
for model in models:
    model_df = pd.read_csv(open(model, 'r'))
    # preprocess model dataframe
    model_df['location'].replace('US', -1, inplace=True)
    model_df = model_df.astype({'location':'int64'})
    merged = model_df.merge(pop[['location', 'Population']], on='location', how='left')
    invalid_preds = np.sum(merged['value'] >= merged['Population'])
    if invalid_preds > 0:
        print(f"Number of invalid predictions for {model.split('/')[-1]} is {invalid_preds}")
    

Number of invalid predictions for 2020-08-31-UVA-Ensemble.csv is 14
Number of invalid predictions for 2020-08-24-UVA-Ensemble.csv is 30
Number of invalid predictions for 2020-09-07-UVA-Ensemble.csv is 2
Number of invalid predictions for 2020-08-03-RobertWalraven-ESG.csv is 44
Number of invalid predictions for 2020-08-01-RobertWalraven-ESG.csv is 40
Number of invalid predictions for 2020-08-10-RobertWalraven-ESG.csv is 24
Number of invalid predictions for 2020-07-26-RobertWalraven-ESG.csv is 93
Number of invalid predictions for 2020-05-11-GT-DeepCOVID.csv is 28
Number of invalid predictions for 2020-09-06-Yu_Group-CLEP.csv is 6
Number of invalid predictions for 2020-08-15-Yu_Group-CLEP.csv is 1
Number of invalid predictions for 2020-09-07-LANL-GrowthRate.csv is 1
Number of invalid predictions for 2020-07-13-LANL-GrowthRate.csv is 1
Number of invalid predictions for 2020-07-20-LANL-GrowthRate.csv is 1
Number of invalid predictions for 2020-04-27-UT-Mobility.csv is 3
Number of invalid pre

In [66]:
for model in models:
    model_df = pd.read_csv(open(model, 'r'))
    # preprocess model dataframe
    model_df['location'].replace('US', -1, inplace=True)
    model_df = model_df.astype({'location':'int64'})
    merged = model_df.merge(pop[['location', 'Population']], on='location', how='left')
    invalid_preds = np.sum(merged['value'] >= merged['Population'])s
    if invalid_preds > 0:
        print(f"Number of invalid predictions for {model.split('/')[-1]} is {invalid_preds}")
        display(merged[merged['value'] >= merged['Population']])

Number of invalid predictions for 2020-08-31-UVA-Ensemble.csv is 14


,forecast_date,target,target_end_date,location,type,quantile,value,Population
49143,2020-08-31,3 wk ahead inc case,2020-09-19,29173,quantile,0.975,10610.699074,10309.0
77022,2020-08-31,4 wk ahead inc case,2020-09-26,48083,quantile,0.900,11535.269904,8175.0
77023,2020-08-31,4 wk ahead inc case,2020-09-26,48083,quantile,0.975,17371.197903,8175.0
80477,2020-08-31,4 wk ahead inc case,2020-09-26,48299,quantile,0.750,27427.051510,21795.0
80478,2020-08-31,4 wk ahead inc case,2020-09-26,48299,quantile,0.900,50048.569356,21795.0
80479,2020-08-31,4 wk ahead inc case,2020-09-26,48299,quantile,0.975,75328.892143,21795.0
80765,2020-08-31,4 wk ahead inc case,2020-09-26,48317,quantile,0.750,13107.859305,5771.0
80766,2020-08-31,4 wk ahead inc case,2020-09-26,48317,quantile,0.900,23908.265278,5771.0
80767,2020-08-31,4 wk ahead inc case,2020-09-26,48317,quantile,0.975,35978.089730,5771.0
81845,2020-08-31,3 wk ahead inc case,2020-09-19,48385,quantile,0.750,6013.282456,3452.0


Number of invalid predictions for 2020-08-24-UVA-Ensemble.csv is 30


,forecast_date,target,target_end_date,location,type,quantile,value,Population
76983,2020-08-24,3 wk ahead inc case,2020-09-12,48081,quantile,0.975,4418.642663,3387.0
77872,2020-08-24,3 wk ahead inc case,2020-09-12,48137,point,NaN,7540.423451,1932.0
77876,2020-08-24,3 wk ahead inc case,2020-09-12,48137,quantile,0.500,7540.423451,1932.0
77877,2020-08-24,3 wk ahead inc case,2020-09-12,48137,quantile,0.750,156357.947094,1932.0
77878,2020-08-24,3 wk ahead inc case,2020-09-12,48137,quantile,0.900,290298.360618,1932.0
77879,2020-08-24,3 wk ahead inc case,2020-09-12,48137,quantile,0.975,439981.370885,1932.0
77880,2020-08-24,4 wk ahead inc case,2020-09-19,48137,point,NaN,2510.278542,1932.0
77884,2020-08-24,4 wk ahead inc case,2020-09-19,48137,quantile,0.500,2510.278542,1932.0
77885,2020-08-24,4 wk ahead inc case,2020-09-19,48137,quantile,0.750,25867.413730,1932.0
77886,2020-08-24,4 wk ahead inc case,2020-09-19,48137,quantile,0.900,46889.564006,1932.0


Number of invalid predictions for 2020-09-07-UVA-Ensemble.csv is 2


...

Number of invalid predictions for 2020-08-10-epiforecasts-ensemble1.csv is 2


,forecast_date,target,target_end_date,location,type,quantile,value,Population
3163,2020-08-10,4 wk ahead inc death,2020-09-05,34,quantile,0.99,43059781,8882190.0
3255,2020-08-10,4 wk ahead cum death,2020-09-05,34,quantile,0.99,43075809,8882190.0


Number of invalid predictions for 2020-08-31-epiforecasts-ensemble1.csv is 2


,forecast_date,target,target_end_date,location,type,quantile,value,Population
91,2020-08-31,4 wk ahead inc death,2020-09-26,1,quantile,0.99,7402557,4903185.0
183,2020-08-31,4 wk ahead cum death,2020-09-26,1,quantile,0.99,7405034,4903185.0


/Users/abdulhannan/anaconda3/envs/covid19-forecast/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
